In [ ]:
!python -m spacy download en_core_web_md

2023-08-18 19:34:58.529864: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-18 19:35:00.032086: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 10.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [ ]:
#Top Ten Closest Vectors using SpaCy model


In [ ]:
import spacy
nlp= spacy.load('en_core_web_md')

In [ ]:
#sun-star+planet=moon      --------------------> equation
from scipy import spatial
cosine_similarity=lambda x,y:1 -spatial.distance.cosine(x,y)

king=nlp.vocab['sun'].vector
man=nlp.vocab['star'].vector
woman=nlp.vocab['planet'].vector
new_vector=king-man+woman

computed_similarities=[]

for word in nlp.vocab:
  if word.has_vector:
    if word.is_lower:
      if word.is_alpha:

        similarity=cosine_similarity(new_vector,word.vector)
        computed_similarities.append((word,similarity))

computed_similarities=sorted(computed_similarities,key=lambda item:-item[1])
print([w[0].text for w in computed_similarities[:10]])

['sun', 'moon', 'planet', 'it', 'fire', 'resting', 'space', 'ice', 'we', 'nothin']


In [ ]:
#top ten closest vectors in the vocabulary to the result of the equation above using Word2Vec model.

In [ ]:
from gensim.models import KeyedVectors
import gensim
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
import gensim.downloader as api
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [ ]:
model_W2V = KeyedVectors.load_word2vec_format(path,binary=True, limit=50000)

In [ ]:
model_W2V.vector_size

300

In [ ]:
model_W2V.most_similar(positive=['sun', 'planets'], negative=['star'])

[('sunlight', 0.5489511489868164),
 ('galaxies', 0.5063886046409607),
 ('equator', 0.497109591960907),
 ('rays', 0.4842822253704071),
 ('Earth', 0.47961288690567017),
 ('Milky_Way', 0.4755062162876129),
 ('cosmos', 0.4662123918533325),
 ('galaxy', 0.4652167856693268),
 ('planetary', 0.46518659591674805),
 ('moons', 0.46118053793907166)]

In [ ]:
#sentiment analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('moviereviews.tsv',sep='\t')
df.head()

,label,review
0,neg,how do films like mouse hunt get into theatres...
1,neg,some talented actresses are blessed with a dem...
2,pos,this has been an extraordinary year for austra...
3,pos,according to hollywood movies made in last few...
4,neg,my first press screening of 1998 and already i...


In [ ]:
df.dropna(inplace=True)

In [ ]:
blanks = []

for i,lb,rv in df.itertuples():
  if type(rv)==str:
    if rv.isspace():
      blanks.append(i)

df.drop(blanks,inplace=True)

In [ ]:
df['label'].value_counts()

neg    969
pos    969
Name: label, dtype: int64

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


sid = SentimentIntensityAnalyzer()
review="the movie was actually pretty good"

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
df['scores'] = df['review'].apply(lambda review: sid.polarity_scores(review))
df.head()

,label,review,scores,compound,comp_score
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg


In [ ]:
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])
df.head()

,label,review,scores,compound
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484


In [ ]:
df['comp_score'] = df['compound'].apply(lambda c: 'pos' if c>=0 else 'neg')

df.head()

,label,review,scores,compound,comp_score
0,neg,how do films like mouse hunt get into theatres...,"{'neg': 0.121, 'neu': 0.778, 'pos': 0.101, 'co...",-0.9125,neg
1,neg,some talented actresses are blessed with a dem...,"{'neg': 0.12, 'neu': 0.775, 'pos': 0.105, 'com...",-0.8618,neg
2,pos,this has been an extraordinary year for austra...,"{'neg': 0.068, 'neu': 0.781, 'pos': 0.15, 'com...",0.9951,pos
3,pos,according to hollywood movies made in last few...,"{'neg': 0.071, 'neu': 0.782, 'pos': 0.147, 'co...",0.9972,pos
4,neg,my first press screening of 1998 and already i...,"{'neg': 0.091, 'neu': 0.817, 'pos': 0.093, 'co...",-0.2484,neg
